# Script to create dengue dataset metadata

# Built environment

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, StringType
from pyspark.sql.functions import year, month, col, sum, udf, substring, split, regexp_replace, when
import glob as gb
#from tqdm import tqdm
import re 
from functools import reduce
import numpy as np
import sys
import glob

In [2]:
sc = SparkContext()
sc.stop()

conf = SparkConf().setAppName("SINAM_dengue_cases")

conf = (conf.setMaster("local[*]")
       .set("spark.executor.memory", "3GB")
       .set("spark.driver.memory", "20GB"))

sc = SparkContext(conf = conf)
spark = SparkSession(sc)
spark

In [3]:
#define paths to files
spark = SparkSession.builder.appName("Dengue").getOrCreate()


# Save columns names of each dengue dataset

In [4]:
path = '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/'
filenames = glob.glob(path + "/*.csv")
filenames

['/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN07(OF).csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN08(OF).csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN09(OF).csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN10(OF).csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN11(OF).csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN12(OF).csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN13(OF)_a.csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN13(OF)_b.csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN14(OF).csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SINAN/DENGON/CSV_format/DENGN15(OF)_A.csv',
 '/Volumes/My Passport/Datasus/Data_lake/Raw/Bancos_SI

In [ ]:
lst_df_columns = []
for f in filenames:
    print(f)
    #df = pd.read_csv(f, low_memory = False, encoding="iso-8859-1")  
    df = spark.read.format("csv").option("header",True).load(f)
    lst_df_columns.append([f[-15:],df.columns])

In [ ]:
# save columns names
#sc.parallelize([lst_df_columns]).saveAsTextFile("/Users/julianeoliveira/Desktop/github/AESOP-Data-Analysis/results/dic")

In [4]:
path = '/Users/julianeoliveira/Downloads/juliane'
filenames = glob.glob(path + "/*.csv")
filenames

['/Users/julianeoliveira/Downloads/juliane/DENGBR21.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR09.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR08.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR20.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR18.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR19.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR00.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR14.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR15.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR01.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR17.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR03.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR02.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR16.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR12.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR06.csv',
 '/Users/julianeoliveira/Downloads/juliane/DENGBR07.csv',
 '/Users/julia

# Check columns values

In [5]:
filenames[0]

'/Users/julianeoliveira/Downloads/juliane/DENGBR21.csv'

In [6]:
df = spark.read.format("csv").option("header",True).load(filenames[0])

In [9]:
df.columns

['TP_NOT',
 'ID_AGRAVO',
 'DT_NOTIFIC',
 'SEM_NOT',
 'NU_ANO',
 'SG_UF_NOT',
 'ID_MUNICIP',
 'ID_REGIONA',
 'ID_UNIDADE',
 'DT_SIN_PRI',
 'SEM_PRI',
 'DT_NASC',
 'NU_IDADE_N',
 'CS_SEXO',
 'CS_GESTANT',
 'CS_RACA',
 'CS_ESCOL_N',
 'SG_UF',
 'ID_MN_RESI',
 'ID_RG_RESI',
 'ID_PAIS',
 'DT_INVEST',
 'ID_OCUPA_N',
 'FEBRE',
 'MIALGIA',
 'CEFALEIA',
 'EXANTEMA',
 'VOMITO',
 'NAUSEA',
 'DOR_COSTAS',
 'CONJUNTVIT',
 'ARTRITE',
 'ARTRALGIA',
 'PETEQUIA_N',
 'LEUCOPENIA',
 'LACO',
 'DOR_RETRO',
 'DIABETES',
 'HEMATOLOG',
 'HEPATOPAT',
 'RENAL',
 'HIPERTENSA',
 'ACIDO_PEPT',
 'AUTO_IMUNE',
 'DT_CHIK_S1',
 'DT_CHIK_S2',
 'DT_PRNT',
 'RES_CHIKS1',
 'RES_CHIKS2',
 'RESUL_PRNT',
 'DT_SORO',
 'RESUL_SORO',
 'DT_NS1',
 'RESUL_NS1',
 'DT_VIRAL',
 'RESUL_VI_N',
 'DT_PCR',
 'RESUL_PCR_',
 'SOROTIPO',
 'HISTOPA_N',
 'IMUNOH_N',
 'HOSPITALIZ',
 'DT_INTERNA',
 'UF',
 'MUNICIPIO',
 'TPAUTOCTO',
 'COUFINF',
 'COPAISINF',
 'COMUNINF',
 'CLASSI_FIN',
 'CRITERIO',
 'DOENCA_TRA',
 'CLINC_CHIK',
 'EVOLUCAO',
 'DT_O

In [11]:
df.select("SOROTIPO").show(5)

+--------+
|SOROTIPO|
+--------+
|    b' '|
|    b' '|
|    b' '|
|    b' '|
|    b' '|
+--------+
only showing top 5 rows



In [15]:
df.select("dt_notific").show(5)

+----------+
|dt_notific|
+----------+
|2007-02-19|
|2007-02-02|
|2007-01-06|
|2007-02-24|
|2007-02-07|
+----------+
only showing top 5 rows



In [14]:
df.select("id_agravo").distinct().show(100)

+---------+
|id_agravo|
+---------+
|     4015|
|     4024|
|     4020|
|     4054|
|     4026|
|     4018|
|     4013|
|     4071|
|     4030|
|     4021|
|     4037|
|     4053|
|     4036|
|     4035|
|     4019|
|     4022|
|     4039|
|     4051|
|     4055|
|     4029|
|     4023|
|     4046|
|     4008|
|     4056|
|     4011|
|     4042|
|     4038|
|     4049|
|     4034|
|     4025|
|     4048|
|     4033|
|     4047|
|     4004|
|      A90|
|     4028|
|     4043|
|     4012|
|     4045|
|     4050|
|     4040|
|     4027|
|     4010|
|     4031|
|     4061|
+---------+

